### PostgreSQL DB for AI-powered Contact Center Intake (AICCI)

CC - Simulate API calls to a PostgreSQL table: https://navex.atlassian.net/browse/AIM-537

* Author:    Kyle Zarifsadr
 * Created:   December 2024

In [1]:
from dotenv import load_dotenv
load_dotenv() 
import numpy as np
import pandas as pd
import json
import os
import openai
import random
import time
from openai import AzureOpenAI
import psycopg
from sentence_transformers import SentenceTransformer
import autogen
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json
from autogen.retrieve_utils import TEXT_FORMATS
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent

/usr/local/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


Reference: https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/contrib/vectordb/pgvectordb

In [2]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "azure",
            "base_url": os.getenv("OPENAI_API_BASE"),
            "api_version": os.getenv("API_VERSION"),
        },
    ],
    "temperature": 0.0,
    "timeout": 300,
}

In [3]:
def embed_openai(text: str):
    client = AzureOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        api_version="2024-02-01",
        azure_endpoint=os.getenv("OPENAI_API_BASE")
    )

    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )

    embeddings = [np.array(embedding.embedding) for embedding in response.data]

    if len(embeddings) == 1:
        return embeddings[0]

    return np.array(embeddings)

In [4]:
conn = psycopg2.connect(
    dbname='test',
    user='postgres',
    password='test_password',
    host='localhost',
    port='5433'
)

cur = conn.cursor()

cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
tables = cur.fetchall()

for table in tables:
    print(table)

for table in tables:
    table_name = table[0]  
    print(f"Dropping table: {table_name}") 
    
    cur.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
    conn.commit() 

cur.close()
conn.close()

print("All tables in 'public' schema have been dropped.")

('guidelines',)
('locations',)
('issuetypes',)
('questions',)
Dropping table: guidelines
Dropping table: locations
Dropping table: issuetypes
Dropping table: questions
All tables in 'public' schema have been dropped.


In [5]:
conn = psycopg2.connect(
    dbname="test",
    user="postgres",
    password="test_password",
    host="localhost",
    port="5433"
)

cur = conn.cursor()

cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
conn.commit()

create_table_query_gui = """
CREATE TABLE IF NOT EXISTS guidelines (
    id SERIAL PRIMARY KEY,
    url TEXT,
    request_data TEXT,
    response_data TEXT,
    clientKey TEXT
);
"""

cur.execute(create_table_query_gui)
conn.commit()

create_table_query_loc = """
CREATE TABLE IF NOT EXISTS locations (
    id SERIAL PRIMARY KEY,
    url TEXT,
    request_data TEXT,
    response_data TEXT,
    clientKey TEXT
);
"""

cur.execute(create_table_query_loc)
conn.commit()

create_table_query_iss = """
CREATE TABLE IF NOT EXISTS issuetypes (
    id SERIAL PRIMARY KEY,
    url TEXT,
    request_data TEXT,
    response_data TEXT,
    clientKey TEXT
);
"""

cur.execute(create_table_query_iss)
conn.commit()

create_table_query_que = """
CREATE TABLE IF NOT EXISTS questions (
    id SERIAL PRIMARY KEY,
    url TEXT,
    request_data TEXT,
    response_data TEXT,
    clientKey TEXT
);
"""

cur.execute(create_table_query_que)
conn.commit()

## Fake Data

### ClientName: "Call Center Training 31 - UPS"

In [6]:
data1={
    "clientKey": "Call Center Training 31 - UPS",
    "CustomText": """Opening

Thank you for calling the Call Center Training 31 - UPS Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report? Finished



CS NOTE: Below is an updated list of companies that are either wholly owned subsidiaries or joint – venture arrangements where the employees may call in a concern. Any calls from employees identifying these companies should be routed to our EPIM dashboard.

Please note, UPS Freight is not included in the listing. The organization was sold to TForce International (TFI) in 2021 and they maintain their own hotline.

United Parcel Service (UPS)
Supply Chain Solutions (SCS)
Mail Innovations
The UPS Store
Coyote
Marken
iParcel
Parcel Pro
Roadie
IRIS Transport
Ware2Go
Movin (India)
Delivery Solutions
BOMI Group
Happy Returns
MNX Global Logistics
READ: Is your question or concern related to “shipper pre-clearance”? YES / NO
READ: Are you reporting a concern related to a package, delivery or UPS account? YES / NO
READ:“In a few words, tell me a little about your concern and then I’ll capture additional details in just a moment.” CS NOTE: Proceed with report. Click here."""
}

In [7]:
data3 = {
  "clientKey": "Call Center Training 31 - UPS",
  "Locations": [
    {
      "Case_CompanyLocation": "Branch Office A",
      "Case_CompanyCity": "Springfield",
      "Case_CompanyState": "IL",
      "Case_CompanyZip": "62701",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Technical Support",
      "Case_LocationCustomField2": "NE",
      "Case_LocationCustomField3": "District 5",
      "Case_LocationCustomField4": "Floor 3"
    },
    {
      "Case_CompanyLocation": "Branch Office B",
      "Case_CompanyCity": "Madison",
      "Case_CompanyState": "WI",
      "Case_CompanyZip": "53703",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Sales",
      "Case_LocationCustomField2": "NW",
      "Case_LocationCustomField3": "District 1",
      "Case_LocationCustomField4": "Building 2"
    },
    {
      "Case_CompanyLocation": "Branch Office C",
      "Case_CompanyCity": "Albany",
      "Case_CompanyState": "NY",
      "Case_CompanyZip": "12207",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Customer Service",
      "Case_LocationCustomField2": "SE",
      "Case_LocationCustomField3": "District 3",
      "Case_LocationCustomField4": "Suite 200"
    }
  ]
}

In [8]:
# GetMobileIssueTypesWithDefaults 

# Request URL
# /v6.0/Intake/GetMobileIssueTypesWithDefaults

# Example Request Body
# {
#     "clientKey": "Call Center Training 31 - UPS",
#     "tierId": 1,
#     "epimApiAuthKey": "U4K@!ugrqQqcrWCV8!IV",
#     "languageCode": "en-US",
#     "locationId": 151127,
#     "locationDataPrivacy": false
# }

# Example Response
data5 = {
    "clientKey": "Call Center Training 31 - UPS",
    "data": {"ViolationTypeList": [
            {
                "Description": "Title IX, Gender/Sexual Identity/Veterans/Religion/Etc., Force customers to split these, Leverage existent binary Discrimination flag, Diversity and Teamwork, Fairness and Equal Opportunity. NA67 - - new customization from Base <br/> Testing",
                "Name": "Discrimination",
               
                "ViolationTypeId": 1
            },
            {
                "Description": "Drug, alcohol or other substance abuse. Drug/Alcohol/Substance, Impairment 1- abuse",
                "Name": "Workplace Civility 1",
               
                "ViolationTypeId": 2
            },
            {
                "Description": "Pay, benefits, bonuses, time off, maternity/paternity leave, vacation , E2E 15/09/2134 - 1234 compansation",
                "Name": "Compensation and Benefits",
              
                "ViolationTypeId": 3
            },
            {
                "Description": "Reports of harassment that are linked to a protected characteristic (such as race, gender, sex, religion, disability, age, etc.) and includes allegations of unwelcome behavior that is offensive to a reasonable person, and is related to, or done because of, a protected characteristic. Integration 1112345",
                "Name": "Harassment",
               
                "ViolationTypeId": 4
            },
            {
                "Description": "Falsified information (non-financial), Healthcare billing and coding. ",
                "Name": "Other Business Integrity",
               
                "ViolationTypeId": 5
            },
            {
                "Description": "Feral cats & donuts - new customization from Base",
                "Name": "Other",
               
                "ViolationTypeId": 6
            },
            {
                "Description": "Reports related to accounting, financial reporting or auditing. Examples include the unethical or improper recording and analysis of the business and financial transactions associated with generally accepted accounting practices. Examples include: misstatement of revenues, misstatement of expenses, misstatement of assets, misapplications of GAAP principles, and wrongful transactions.<br><b>test testinghtml</b><br><ul><li>one<li>two</ul><br><p><b>This text is bold</b></p>\r\n<p><i>This text is italic</i></p>\r\n<p>This is<sub> subscript</sub> and <sup>superscript</sup></p>",
                "Name": "Accounting, Auditing and Financial Reporting_test",
               
                "ViolationTypeId": 7
            },
            {
                "Description": "Reports related to confidential and proprietary information or intellectual property. Confidential information is any non-public information that is not intended or permitted to be shared beyond those with a genuine business need to know the information. Confidential information can include information about people or companies and specifically includes business plans, trade secret information, customer lists, sales and marketing strategies, pricing, product development plans, and any notes or documentation of the foregoing. Intellectual property refers to an original, intangible creation of human intellect that is legally protected from unauthorized use. Intellectual property includes patents, trademarks and copyrighted works of authorship, like photographs, music, literary works, graphic design, source code, and audio and audiovisual recordings.",
                "Name": "Confidential and Proprietary Information",
                
                "ViolationTypeId": 8
            },
            {
                "Description": "Reports related to accounting, financial reporting or auditing. Examples include the unethical or improper recording and analysis of the business and financial transactions associated with generally accepted accounting practices. Examples include: misstatement of revenues, misstatement of expenses, misstatement of assets, misapplications of GAAP principles, and wrongful transactions.",
                "Name": "Accounting, Auditing and Financial Reporting",
               
                "ViolationTypeId": 9
            },
            {
                "Description": "Reports of public or private instances of bribery. Bribery occurs when a person offers money or something else of value – to an official or someone in a position of power or influence – for the purpose of gaining influence over them. Corruption includes dishonest or illegal behavior – especially of people in authority – using their power to do dishonest or <b>illegal things</b> in return for money or to get an advantage over someone else.\r\n<h1> test</h1>",
                "Name": "Bribery and Corruption",
               
                "ViolationTypeId": 10
            },
            {
                "Description": "Reports related to abusive or disrespectful behavior connected to work that are not harassment or discrimination. ",
                "Name": "Workplace Civility",
              
                "ViolationTypeId": 11
            },
            {
                "Description": "Reports of imminent or immediate threat of harm to a person or people, animals or property. Reports may or may not involve a weapon and generally are the kind of incident where authorities (such as police or fire) are called to assist.",
                "Name": "Imminent Threat to a Person, Animals or Property",
               
                "ViolationTypeId": 12
            }
          ]
    }
}


In [9]:
data7 = {
  "clientKey": "Call Center Training 31 - UPS",
  "data": [
    {
      "violationTypeId": 2,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 2,
      "ViolationQuestionId": 138,
      "Question": "What substance was involved?"
    },
    {
      "violationTypeId": 2,
      "ViolationQuestionId": 140,
      "Question": "Who was under the influence?"
    },
    
    {
      "violationTypeId": 3,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 3,
      "ViolationQuestionId": 142,
      "Question": "What benefits were affected?"
    },
    {
      "violationTypeId": 3,
      "ViolationQuestionId": 143,
      "Question": "When did this issue occur?"
    },
    
    {
      "violationTypeId": 4,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 4,
      "ViolationQuestionId": 145,
      "Question": "What type of harassment occurred?"
    },
    {
      "violationTypeId": 4,
      "ViolationQuestionId": 146,
      "Question": "Who was harassed?"
    },
    
    {
      "violationTypeId": 5,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 5,
      "ViolationQuestionId": 148,
      "Question": "What information was falsified?"
    },
    {
      "violationTypeId": 5,
      "ViolationQuestionId": 149,
      "Question": "Who was involved in falsifying information?"
    },
    
    {
      "violationTypeId": 6,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 6,
      "ViolationQuestionId": 151,
      "Question": "What was the nature of the incident?"
    },
    {
      "violationTypeId": 6,
      "ViolationQuestionId": 137,
      "Question": "Who was involved in the incident?"
    },
    
    {
      "violationTypeId": 7,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 7,
      "ViolationQuestionId": 154,
      "Question": "What type of financial misconduct occurred?"
    },
    {
      "violationTypeId": 7,
      "ViolationQuestionId": 155,
      "Question": "Who was involved in the financial misconduct?"
    },
    
    {
      "violationTypeId": 8,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 8,
      "ViolationQuestionId": 157,
      "Question": "What type of confidential information was involved?"
    },
    {
      "violationTypeId": 8,
      "ViolationQuestionId": 158,
      "Question": "Who had access to the confidential information?"
    },
    
    {
      "violationTypeId": 9,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 9,
      "ViolationQuestionId": 154,
      "Question": "What type of financial misconduct occurred?"
    },
    {
      "violationTypeId": 9,
      "ViolationQuestionId": 155,
      "Question": "Who was involved in the financial misconduct?"
    },
    
    {
      "violationTypeId": 10,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 10,
      "ViolationQuestionId": 163,
      "Question": "What was the nature of the bribery or corruption?"
    },
    {
      "violationTypeId": 10,
      "ViolationQuestionId": 164,
      "Question": "Who was involved in the bribery or corruption?"
    },
    
    {
      "violationTypeId": 11,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 11,
      "ViolationQuestionId": 166,
      "Question": "What type of disrespectful behavior occurred?"
    },
    {
      "violationTypeId": 11,
      "ViolationQuestionId": 167,
      "Question": "Who was involved in the disrespectful behavior?"
    },
    
    {
      "violationTypeId": 12,
      "ViolationQuestionId": 135,
      "Question": "Where did this happen?"
    },
    {
      "violationTypeId": 12,
      "ViolationQuestionId": 169,
      "Question": "What was the nature of the threat?"
    },
    {
      "violationTypeId": 12,
      "ViolationQuestionId": 170,
      "Question": "Who was involved in the threat?"
    }
  ]
}

### ClientName: "Call Center Training 32 - Target Corporation"

In [10]:
data2 = {
    "clientKey": "Call Center Training 32 - Target Corporation",
    "CustomText": """
Opening

Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report? Finished

DON'T READ: Factory workers may report on this line. Accept reports from factory workers who work for or with Call Center Training 32 - Target Corporation.
READ: Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I’ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish to proceed?

DON'T READ: Please accept reports from Starbucks employees that work inside Target retail stores.
DON'T READ: Please accept reports from SHIPT employees.
DON'T READ: Do not accept reports for CVS employees.
"""}

In [11]:
data4 = {
  "clientKey": "Call Center Training 32 - Target Corporation",
  "Locations": [
    {
      "Case_CompanyLocation": "Branch Office D",
      "Case_CompanyCity": "Austin",
      "Case_CompanyState": "TX",
      "Case_CompanyZip": "73301",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Technical Support",
      "Case_LocationCustomField2": "SW",
      "Case_LocationCustomField3": "District 7",
      "Case_LocationCustomField4": "Floor 5"
    },
    {
      "Case_CompanyLocation": "Branch Office E",
      "Case_CompanyCity": "Denver",
      "Case_CompanyState": "CO",
      "Case_CompanyZip": "80201",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Sales",
      "Case_LocationCustomField2": "MW",
      "Case_LocationCustomField3": "District 4",
      "Case_LocationCustomField4": "Building 1"
    },
    {
      "Case_CompanyLocation": "Branch Office F",
      "Case_CompanyCity": "Orlando",
      "Case_CompanyState": "FL",
      "Case_CompanyZip": "32801",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Customer Service",
      "Case_LocationCustomField2": "SE",
      "Case_LocationCustomField3": "District 2",
      "Case_LocationCustomField4": "Suite 300"
    }
  ]
}

In [12]:
# GetMobileIssueTypesWithDefaults 

# Request URL
# /v6.0/Intake/GetMobileIssueTypesWithDefaults

# Example Request Body
# {
#     "clientKey": "Call Center Training 32 - Target Corporation",
#     "tierId": 2,
#     "epimApiAuthKey": "U4KsdggQqcrWCV8!IV",
#     "languageCode": "en-US",
#     "locationId": 151129,
#     "locationDataPrivacy": false
# }

# Example Response
data6 = {
    "clientKey": "Call Center Training 32 - Target Corporation",
    "data": {
        "ViolationTypeList": [
            {
                "Description": "Unauthorized access to company systems or data, hacking, or other cyber threats. This includes phishing attempts, malware infections, and other cybersecurity incidents.",
                "Name": "Cybersecurity Breach",
                "ViolationTypeId": 13
            },
            {
                "Description": "Violations related to environmental regulations, including improper disposal of hazardous materials, pollution, and non-compliance with environmental laws.",
                "Name": "Environmental Violation",
                "ViolationTypeId": 14
            },
            {
                "Description": "Reports of physical violence or threats of violence in the workplace. This includes fights, assaults, and other forms of physical aggression.",
                "Name": "Workplace Violence",
                "ViolationTypeId": 15
            },
            {
                "Description": "Instances of theft, embezzlement, or other forms of financial fraud. This includes stealing company property, misappropriating funds, and falsifying financial records.",
                "Name": "Financial Fraud",
                "ViolationTypeId": 16
            },
            {
                "Description": "Violations related to health and safety regulations, including unsafe working conditions, lack of proper safety equipment, and failure to follow safety protocols.",
                "Name": "Health and Safety Violation",
                "ViolationTypeId": 17
            },
            {
                "Description": "Reports of discrimination based on age, including unfair treatment, harassment, or bias against employees or customers due to their age.",
                "Name": "Age Discrimination",
                "ViolationTypeId": 18
            },
            {
                "Description": "Instances of unethical behavior or conflicts of interest, such as accepting gifts or favors in exchange for preferential treatment or using company resources for personal gain.",
                "Name": "Ethical Violation",
                "ViolationTypeId": 19
            },
            {
                "Description": "Reports of violations related to data privacy, including unauthorized sharing of personal information, data breaches, and failure to comply with data protection regulations.",
                "Name": "Data Privacy Violation",
                "ViolationTypeId": 20
            },
            {
                "Description": "Instances of workplace bullying, including repeated mistreatment, verbal abuse, and other forms of hostile behavior that create a toxic work environment.",
                "Name": "Workplace Bullying",
                "ViolationTypeId": 21
            },
            {
                "Description": "Reports of non-compliance with legal or regulatory requirements, including violations of industry standards, government regulations, and company policies.",
                "Name": "Regulatory Non-Compliance",
                "ViolationTypeId": 22
            }
        ]
    }
}

In [13]:
data8 = {
        "clientKey": "Call Center Training 32 - Target Corporation", 
        "data": [
            {
                "violationTypeId": 13,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 13,
                "ViolationQuestionId": 172,
                "Question": "What type of cyber threat occurred?"
            },
            {
                "violationTypeId": 13,
                "ViolationQuestionId": 173,
                "Question": "Who was involved in the cybersecurity breach?"
            },
    
            {
                "violationTypeId": 14,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 14,
                "ViolationQuestionId": 174,
                "Question": "What environmental regulation was violated?"
            },
            {
                "violationTypeId": 14,
                "ViolationQuestionId": 175,
                "Question": "Who was responsible for the environmental violation?"
            },
    
            {
                "violationTypeId": 15,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 15,
                "ViolationQuestionId": 176,
                "Question": "What type of violence occurred?"
            },
            {
                "violationTypeId": 15,
                "ViolationQuestionId": 177,
                "Question": "Who was involved in the workplace violence?"
            },
    
            {
                "violationTypeId": 16,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 16,
                "ViolationQuestionId": 178,
                "Question": "What type of financial fraud occurred?"
            },
            {
                "violationTypeId": 16,
                "ViolationQuestionId": 179,
                "Question": "Who was involved in the financial fraud?"
            },
    
            {
                "violationTypeId": 17,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 17,
                "ViolationQuestionId": 180,
                "Question": "What health and safety regulation was violated?"
            },
            {
                "violationTypeId": 17,
                "ViolationQuestionId": 181,
                "Question": "Who was responsible for the health and safety violation?"
            },
    
            {
                "violationTypeId": 18,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 18,
                "ViolationQuestionId": 182,
                "Question": "What type of age discrimination occurred?"
            },
            {
                "violationTypeId": 18,
                "ViolationQuestionId": 183,
                "Question": "Who was involved in the age discrimination?"
            },
    
            {
                "violationTypeId": 19,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 19,
                "ViolationQuestionId": 184,
                "Question": "What type of ethical violation occurred?"
            },
            {
                "violationTypeId": 19,
                "ViolationQuestionId": 185,
                "Question": "Who was involved in the ethical violation?"
            },
    
            {
                "violationTypeId": 20,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 20,
                "ViolationQuestionId": 186,
                "Question": "What type of data privacy violation occurred?"
            },
            {
                "violationTypeId": 20,
                "ViolationQuestionId": 187,
                "Question": "Who was involved in the data privacy violation?"
            },
    
            {
                "violationTypeId": 21,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 21,
                "ViolationQuestionId": 188,
                "Question": "What type of workplace bullying occurred?"
            },
            {
                "violationTypeId": 21,
                "ViolationQuestionId": 189,
                "Question": "Who was involved in the workplace bullying?"
            },
    
            {
                "violationTypeId": 22,
                "ViolationQuestionId": 171,
                "Question": "Where did this happen?"
            },
            {
                "violationTypeId": 22,
                "ViolationQuestionId": 190,
                "Question": "What type of regulatory non-compliance occurred?"
            },
            {
                "violationTypeId": 22,
                "ViolationQuestionId": 191,
                "Question": "Who was involved in the regulatory non-compliance?"
            }
        ]
    }

In [14]:
def generate_key(index):
    return f"{index}"

def generate_fake_request_data():
    return {
        "request_id": random.randint(1000, 9999),
        "request_type": "GET"
    }

def create_dataframe_gui(data, index):
    custom_text_str = json.dumps(data["CustomText"])

    df = pd.DataFrame({
        "id": [generate_key(index)],
        "url": '/v6.0/Intake/guidelines',
        "request_data": [json.dumps(generate_fake_request_data())],
        "response_data": [json.dumps(data)],
        "clientKey": [data["clientKey"]]
    })

    return df

def create_dataframe_loc(data, index):
    custom_text_str = json.dumps(data["Locations"])

    df = pd.DataFrame({
        "id": [generate_key(index)],
        "url": '/v6.0/Intake/locations',
        "request_data": [json.dumps(generate_fake_request_data())],
        "response_data": [json.dumps(data)],
        "clientKey": [data["clientKey"]]
    })

    return df

def create_dataframe_iss(data, index):
    custom_text_str = json.dumps(data["data"])

    df = pd.DataFrame({
        "id": [generate_key(index)],
        "url": '/v6.0/Intake/GetMobileIssueTypesWithDefaults',
        "request_data": [json.dumps(generate_fake_request_data())],
        "response_data": [json.dumps(data)],
        "clientKey": [data["clientKey"]]
    })

    return df


def create_dataframe_que(data, index):
    custom_text_str = json.dumps(data["data"])

    df = pd.DataFrame({
        "id": [generate_key(index)],
        "url": '/v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages',
        "request_data": [json.dumps(generate_fake_request_data())],
        "response_data": [json.dumps(data)],
        "clientKey": [data["clientKey"]]
    })

    return df

In [15]:
df1 = create_dataframe_gui(data1, 1)
df2 = create_dataframe_gui(data2, 2)

df3 = create_dataframe_loc(data3, 3)
df4 = create_dataframe_loc(data4, 4)

df5 = create_dataframe_iss(data5, 5)
df6 = create_dataframe_iss(data6, 6)

df7 = create_dataframe_que(data7, 7)
df8 = create_dataframe_que(data8, 8)

In [16]:
def insert_data_from_dataframe(df, cur):
    data_list_gui = [
        (
            row["url"], 
            row["request_data"], 
            row["response_data"], 
            row["clientKey"]
        )
        for index, row in df.iterrows()
    ]

    insert_query_gui = """
    INSERT INTO guidelines (url, request_data, response_data, clientKey) 
    VALUES %s
    ON CONFLICT (id) DO UPDATE 
    SET 
        url = EXCLUDED.url,
        request_data = EXCLUDED.request_data,
        response_data = EXCLUDED.response_data,
        clientKey = EXCLUDED.clientKey;
    """

    execute_values(cur, insert_query_gui, data_list_gui)

    data_list_loc = [
        (
            row["url"], 
            row["request_data"], 
            row["response_data"], 
            row["clientKey"]
        )
        for index, row in df.iterrows()
    ]

    insert_query_loc = """
    INSERT INTO locations (url, request_data, response_data, clientKey) 
    VALUES %s
    ON CONFLICT (id) DO UPDATE 
    SET 
        url = EXCLUDED.url,
        request_data = EXCLUDED.request_data,
        response_data = EXCLUDED.response_data,
        clientKey = EXCLUDED.clientKey;
    """

    execute_values(cur, insert_query_loc, data_list_loc)

    data_list_iss = [
        (
            row["url"], 
            row["request_data"], 
            row["response_data"], 
            row["clientKey"]
        )
        for index, row in df.iterrows()
    ]

    insert_query_iss = """
    INSERT INTO issuetypes (url, request_data, response_data, clientKey) 
    VALUES %s
    ON CONFLICT (id) DO UPDATE 
    SET 
        url = EXCLUDED.url,
        request_data = EXCLUDED.request_data,
        response_data = EXCLUDED.response_data,
        clientKey = EXCLUDED.clientKey;
    """

    execute_values(cur, insert_query_iss, data_list_iss)

    data_list_que = [
        (
            row["url"], 
            row["request_data"], 
            row["response_data"], 
            row["clientKey"]
        )
        for index, row in df.iterrows()
    ]

    insert_query_que = """
    INSERT INTO Questions (url, request_data, response_data, clientKey) 
    VALUES %s
    ON CONFLICT (id) DO UPDATE 
    SET 
        url = EXCLUDED.url,
        request_data = EXCLUDED.request_data,
        response_data = EXCLUDED.response_data,
        clientKey = EXCLUDED.clientKey;
    """

    execute_values(cur, insert_query_que, data_list_que)


conn = psycopg2.connect(
    dbname="test",
    user="postgres",
    password="test_password",
    host="localhost",
    port="5433"
)

cur = conn.cursor()

dataframes = [df1, df2, df3, df4, df5, df6, df7, df8]

try:
    for i, df in enumerate(dataframes):
        insert_data_from_dataframe(df, cur)
    
    conn.commit()  
except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

print(f"Data from {len(dataframes)} DataFrames inserted or updated successfully.")

Data from 8 DataFrames inserted or updated successfully.


### Code to Simulate API Calls to PostgreSQL

In [17]:
def get_db_connection():
    conn = psycopg2.connect(
        dbname="test",
        user="postgres",
        password="test_password",
        host="localhost",
        port="5433"
    )
    return conn

def make_request(url, clientKey):
    conn = get_db_connection()
    cur = conn.cursor()

    query = "SELECT response_data FROM guidelines WHERE url = %s AND clientKey = %s;"
    cur.execute(query, (url, clientKey))
    result = cur.fetchone()
    
    if result:
        response_data = result[0]
        print(f"Response from {url}: {json.dumps(response_data, indent=2)}")
    #     store_request_response(url, params, response_data)
    # else:
    #     print(f"No data found for URL: {url}")
    #     store_request_response(url, params, {"error": "Not Found"})
    
    cur.close()
    conn.close()

# def store_request_response(url, request_data, response_data):
#     conn = get_db_connection()
#     cur = conn.cursor()

#     try:
#         insert_query = """
#         INSERT INTO guidelines (url, request_data, response_data)
#         VALUES (%s, %s, %s)
#         ON CONFLICT (url) DO UPDATE SET
#         request_data = EXCLUDED.request_data,
#         response_data = EXCLUDED.response_data
#         """
#         cur.execute(insert_query, (url, json.dumps(request_data), json.dumps(response_data)))
#         conn.commit()
#     except Exception as e:
#         print(f"Error storing request response: {e}")
#     finally:
#         cur.close()
#         conn.close()

In [18]:
url = "/v6.0/Intake/guidelines"
clientKey = "Call Center Training 32 - Target Corporation"
response = make_request(url, clientKey)
print(response)

Response from /v6.0/Intake/guidelines: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"CustomText\": \"\\nOpening\\n\\nThank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report? Finished\\n\\nDON'T READ: Factory workers may report on this line. Accept reports from factory workers who work for or with Call Center Training 32 - Target Corporation.\\nREAD: Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I\\u2019ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish to proceed?\\n\\nDON'T READ: Please accept reports from Starbucks employees that work inside Target retail stores.\\nDON'T READ: Please accept reports from SHIPT 

In [19]:
url = "/v6.0/Intake/locations"
clientKey = "Call Center Training 32 - Target Corporation"
response = make_request(url, clientKey)
print(response)

Response from /v6.0/Intake/locations: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"Locations\": [{\"Case_CompanyLocation\": \"Branch Office D\", \"Case_CompanyCity\": \"Austin\", \"Case_CompanyState\": \"TX\", \"Case_CompanyZip\": \"73301\", \"Case_CompanyCountry\": \"USA\", \"Case_LocationCustomField1\": \"Technical Support\", \"Case_LocationCustomField2\": \"SW\", \"Case_LocationCustomField3\": \"District 7\", \"Case_LocationCustomField4\": \"Floor 5\"}, {\"Case_CompanyLocation\": \"Branch Office E\", \"Case_CompanyCity\": \"Denver\", \"Case_CompanyState\": \"CO\", \"Case_CompanyZip\": \"80201\", \"Case_CompanyCountry\": \"USA\", \"Case_LocationCustomField1\": \"Sales\", \"Case_LocationCustomField2\": \"MW\", \"Case_LocationCustomField3\": \"District 4\", \"Case_LocationCustomField4\": \"Building 1\"}, {\"Case_CompanyLocation\": \"Branch Office F\", \"Case_CompanyCity\": \"Orlando\", \"Case_CompanyState\": \"FL\", \"Case_CompanyZip\": \"32801\", \"Case_Company

In [20]:
url = "/v6.0/Intake/GetMobileIssueTypesWithDefaults"
clientKey = "Call Center Training 32 - Target Corporation"
response = make_request(url, clientKey)
print(response)

Response from /v6.0/Intake/GetMobileIssueTypesWithDefaults: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"data\": {\"ViolationTypeList\": [{\"Description\": \"Unauthorized access to company systems or data, hacking, or other cyber threats. This includes phishing attempts, malware infections, and other cybersecurity incidents.\", \"Name\": \"Cybersecurity Breach\", \"ViolationTypeId\": 13}, {\"Description\": \"Violations related to environmental regulations, including improper disposal of hazardous materials, pollution, and non-compliance with environmental laws.\", \"Name\": \"Environmental Violation\", \"ViolationTypeId\": 14}, {\"Description\": \"Reports of physical violence or threats of violence in the workplace. This includes fights, assaults, and other forms of physical aggression.\", \"Name\": \"Workplace Violence\", \"ViolationTypeId\": 15}, {\"Description\": \"Instances of theft, embezzlement, or other forms of financial fraud. This includes stealing com

In [21]:
url = "/v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages"
clientKey = "Call Center Training 32 - Target Corporation"
response = make_request(url, clientKey)
print(response)

Response from /v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"data\": [{\"violationTypeId\": 13, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 13, \"ViolationQuestionId\": 172, \"Question\": \"What type of cyber threat occurred?\"}, {\"violationTypeId\": 13, \"ViolationQuestionId\": 173, \"Question\": \"Who was involved in the cybersecurity breach?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 174, \"Question\": \"What environmental regulation was violated?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 175, \"Question\": \"Who was responsible for the environmental violation?\"}, {\"violationTypeId\": 15, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 15, \"ViolationQuestionId\": 17

### Retrieve with "ClientName"

In [22]:
def execute_query(query: str) -> str: 
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("MYPOSTGRES_DATABASE"),
            user=os.getenv("MYPOSTGRES_USER"),
            password=os.getenv("MYPOSTGRES_PASSWORD"), 
            host=os.getenv("MYPOSTGRES_HOST"),
            port=os.getenv("MYPOSTGRES_PORT")
        )

        register_vector(conn)
        cur = conn.cursor()
        cur.execute(query)
        res = cur.fetchall()
        cur.close()
        conn.close()
        return res
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


execute_query("SELECT response_data FROM guidelines WHERE url = '/v6.0/Intake/guidelines' and clientKey = 'Call Center Training 31 - UPS';")

[('{"clientKey": "Call Center Training 31 - UPS", "CustomText": "Opening\\n\\nThank you for calling the Call Center Training 31 - UPS Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report? Finished\\n\\n\\n\\nCS NOTE: Below is an updated list of companies that are either wholly owned subsidiaries or joint \\u2013 venture arrangements where the employees may call in a concern. Any calls from employees identifying these companies should be routed to our EPIM dashboard.\\n\\nPlease note, UPS Freight is not included in the listing. The organization was sold to TForce International (TFI) in 2021 and they maintain their own hotline.\\n\\nUnited Parcel Service (UPS)\\nSupply Chain Solutions (SCS)\\nMail Innovations\\nThe UPS Store\\nCoyote\\nMarken\\niParcel\\nParcel Pro\\nRoadie\\nIRIS Transport\\nWare2Go\\nMovin (India)\\nDelivery Solutions\\nBOMI Group\\nHappy Returns\\nMNX Global Logistics\\nREAD: Is your question or concern related to \\u201cshipp

## Text to PostgreSQL SQL with Groupchat 

### Tool Schema
Pydantic model

https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use#tool-schema

### Tool 1

In [23]:
def sql_writer() -> str: 

    conn = psycopg2.connect(
        dbname=os.getenv("MYPOSTGRES_DATABASE"),
        user=os.getenv("MYPOSTGRES_USER"),
        password=os.getenv("MYPOSTGRES_PASSWORD"), 
        host=os.getenv("MYPOSTGRES_HOST"),
        port=os.getenv("MYPOSTGRES_PORT")
    )

    cur = conn.cursor()

    query = """
    SELECT 
        table_schema, 
        table_name, 
        column_name, 
        data_type
    FROM 
        information_schema.columns
    WHERE 
        table_schema NOT IN ('information_schema', 'pg_catalog')  -- Exclude system schemas
    ORDER BY 
        table_schema, 
        table_name, 
        ordinal_position;
    """

    cur.execute(query)

    rows = cur.fetchall()

    schema_info = {}
    for row in rows:
        table_schema, table_name, column_name, data_type = row
        if table_schema not in schema_info:
            schema_info[table_schema] = {}
        if table_name not in schema_info[table_schema]:
            schema_info[table_schema][table_name] = []
        schema_info[table_schema][table_name].append({
            "column_name": column_name,
            "data_type": data_type
        })

    cur.close()
    conn.close()  

    return json.dumps(schema_info, indent=4)

json_columns_info = sql_writer()
print(json_columns_info)

{
    "public": {
        "guidelines": [
            {
                "column_name": "id",
                "data_type": "integer"
            },
            {
                "column_name": "url",
                "data_type": "text"
            },
            {
                "column_name": "request_data",
                "data_type": "text"
            },
            {
                "column_name": "response_data",
                "data_type": "text"
            },
            {
                "column_name": "clientkey",
                "data_type": "text"
            }
        ],
        "issuetypes": [
            {
                "column_name": "id",
                "data_type": "integer"
            },
            {
                "column_name": "url",
                "data_type": "text"
            },
            {
                "column_name": "request_data",
                "data_type": "text"
            },
            {
                "column_name": "response_data",


### Tool 2

In [24]:
def execute_query(query: str) -> str: 
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("MYPOSTGRES_DATABASE"),
            user=os.getenv("MYPOSTGRES_USER"),
            password=os.getenv("MYPOSTGRES_PASSWORD"), 
            host=os.getenv("MYPOSTGRES_HOST"),
            port=os.getenv("MYPOSTGRES_PORT")
        )

        register_vector(conn)
        cur = conn.cursor()
        cur.execute(query)
        res = cur.fetchall()
        cur.close()
        conn.close()
        return res
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Agent 1

In [25]:
template1 = """
Role: PostgreSQL SQL Writer Specialist
Task: Given a question and schema information you receive using the tool sql_writer, your job is to create a PostgreSQL SQL query
based on the question requirements.
Output: Valid PostgreSQL SQL query (with proper joins, filtering, grouping, etc., based on the given goal).
Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```)
"""

In [26]:
text_to_sql_agent = ConversableAgent(
    name="text_to_sql_agent",
    system_message= template1 + "schema information:",
    llm_config=llm_config,
)

### Agent 2

In [27]:
template2 = """
PostgreSQL SQL Executor Task: Given a PostgreSQL SQL query you receive,
your job is to execute the PostgreSQL SQL query using the tool execute_query.
Output: Valid results
Example: execute_query("SELECT metadatas FROM public.policies;")
Results: 
[('Driver Education Course Requirement',),
 ('Behind-the-Wheel Training',),
 ('License Renewal Training',),
 ('Advanced Driving Course',),
 ('Teen Driver Safety Training',)]

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```)
"""

In [28]:
execute_sql_agent = ConversableAgent(
    name="execute_sql_agent",
    system_message= template2 + "Query:",
    llm_config=llm_config,
)

### Agent 3

In [29]:
user = ConversableAgent(
    name="user",
    system_message= "You are asking questions to your assistants and reporting back in a detailed tone.",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

### Registering Tools

In [30]:
text_to_sql_agent.register_for_llm(
    name="sql_writer", 
    description="Generate PostgreSQL SQL query."
)(sql_writer)


execute_sql_agent.register_for_llm(
    name="execute_query",
    description= "Execute a PostgreSQL SQL query."
)(execute_query)


user.register_for_execution(
    name="sql_writer"
)(sql_writer)


execute_sql_agent.register_for_execution(
    name="execute_query"
)(execute_query)

<function __main__.execute_query(query: str) -> str>

### Group Chat

In [31]:
allowed_transitions = {
    user: [text_to_sql_agent],
    text_to_sql_agent: [execute_sql_agent],
    execute_sql_agent: [user]
}

constrained_graph_chat = GroupChat(
    agents=[user,
            text_to_sql_agent,
            execute_sql_agent
           ],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=10,
    send_introductions=True,
)

constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config=llm_config,
)

## Question

In [32]:
user_query = "Give me the IssueType informations about 'Call Center Training 32 - Target Corporation'?"

## initiate_chat

In [33]:
chat_result = user.initiate_chat(
    constrained_group_chat_manager,
    message= f"question: {user_query}",
    summary_method="last_msg",
)

user (to chat_manager):

question: Give me the IssueType informations about 'Call Center Training 32 - Target Corporation'?

--------------------------------------------------------------------------------

Next speaker: text_to_sql_agent


>>>>>>>> USING AUTO REPLY...
text_to_sql_agent (to chat_manager):

***** Suggested tool call (call_4QXv3u6rBlqYdBlJLIS32xSv): sql_writer *****
Arguments: 
{}
***************************************************************************

--------------------------------------------------------------------------------

Next speaker: user


>>>>>>>> EXECUTING FUNCTION sql_writer...
user (to chat_manager):

user (to chat_manager):

***** Response from calling tool (call_4QXv3u6rBlqYdBlJLIS32xSv) *****
{
    "public": {
        "guidelines": [
            {
                "column_name": "id",
                "data_type": "integer"
            },
            {
                "column_name": "url",
                "data_type": "text"
            },
        